### https://medium.com/@patelneha1495/recommendation-system-in-python-using-als-algorithm-and-apache-spark-27aca08eaab3

In [1]:
spark

In [2]:
sc

<SparkContext master=spark://devenv:7077 appName=PySparkShell>

In [3]:
df = spark.read.csv("hdfs://devenv/user/spark/recommendation_system/data/ratings.csv",header=True,
                   schema="COMMODITY_DESC string, household_key string, QUANTITY Integer,\
                   BASKET_ID string, rating float, rating_label Integer") 

In [4]:
df_new = df.select(df['COMMODITY_DESC'],df['household_key'],df['rating_label'])

In [5]:
df_new.show()

+--------------------+-------------+------------+
|      COMMODITY_DESC|household_key|rating_label|
+--------------------+-------------+------------+
|              YOGURT|         2500|           5|
|WATER - CARBONATE...|         2500|           5|
|    WAREHOUSE SNACKS|         2500|           5|
|            VITAMINS|         2500|           5|
|    VEGETABLES SALAD|         2500|           3|
|VEGETABLES - SHEL...|         2500|           5|
|VEGETABLES - ALL ...|         2500|           5|
|   VALUE ADDED FRUIT|         2500|           4|
|           VALENTINE|         2500|           5|
|      TROPICAL FRUIT|         2500|           5|
|            TOMATOES|         2500|           4|
|             TICKETS|         2500|           4|
|                TEAS|         2500|           5|
|     SYRUPS/TOPPINGS|         2500|           3|
|    SUGARS/SWEETNERS|         2500|           5|
|         STONE FRUIT|         2500|           5|
|STATIONERY & SCHO...|         2500|           5|


In [6]:
import pandas as pd
import numpy as np

In [7]:
data = pd.DataFrame(df_new.collect()).rename(columns={0:'COMMODITY_DESC',1:'household_key',2:'rating_label'})

In [8]:
df_matrix = pd.pivot_table(data, values='rating_label', index='household_key', columns='COMMODITY_DESC')

df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [9]:
# create a table for input to the modeling  

d = df_matrix_norm.reset_index() 

d.index.names = ['scaled_rating_label'] 

data_norm = pd.melt(d, id_vars=['household_key'], value_name='scaled_rating_label').dropna()

print(data_norm.shape)

data_norm.head()



(284168, 3)


,household_key,COMMODITY_DESC,scaled_rating_label
3,1000,ADULT INCONTINENCE,0.75
37,1031,ADULT INCONTINENCE,1.00
59,1051,ADULT INCONTINENCE,0.75
75,1066,ADULT INCONTINENCE,0.00
110,1098,ADULT INCONTINENCE,1.00


In [10]:
data_norm = spark.createDataFrame(data_norm)

In [11]:
data_norm.show()

+-------------+------------------+-------------------+
|household_key|    COMMODITY_DESC|scaled_rating_label|
+-------------+------------------+-------------------+
|         1000|ADULT INCONTINENCE|               0.75|
|         1031|ADULT INCONTINENCE|                1.0|
|         1051|ADULT INCONTINENCE|               0.75|
|         1066|ADULT INCONTINENCE|                0.0|
|         1098|ADULT INCONTINENCE|                1.0|
|         1130|ADULT INCONTINENCE|               0.25|
|         1137|ADULT INCONTINENCE|                1.0|
|         1179|ADULT INCONTINENCE|               0.75|
|         1185|ADULT INCONTINENCE|                1.0|
|          120|ADULT INCONTINENCE|                1.0|
|         1236|ADULT INCONTINENCE|               0.25|
|          125|ADULT INCONTINENCE|               0.25|
|         1274|ADULT INCONTINENCE|                1.0|
|         1299|ADULT INCONTINENCE|                1.0|
|         1318|ADULT INCONTINENCE|                1.0|
|         

### Importing important modules

In [12]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

### Converting String to index 
- Before making an ALS model it needs to be clear that ALS only accepts integer value as parameters. Hence we need to convert asin and reviewerID column in index form.
- StringIndexer 
 - MLlib method
 - A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values.

In [13]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

# encoding
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(data_norm.columns)-set(['rating_label']))]

# encoding 簡單一點的寫法
#indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in ['COMMODITY_DESC','household_key']]


# make pipeline
pipeline = Pipeline(stages=indexer)

# fit and transform
transformed_norm= pipeline.fit(data_norm).transform(data_norm)

transformed_norm.show()

+-------------+------------------+-------------------+--------------------+-------------------+-------------------------+
|household_key|    COMMODITY_DESC|scaled_rating_label|COMMODITY_DESC_index|household_key_index|scaled_rating_label_index|
+-------------+------------------+-------------------+--------------------+-------------------+-------------------------+
|         1000|ADULT INCONTINENCE|               0.75|               251.0|              987.0|                      1.0|
|         1031|ADULT INCONTINENCE|                1.0|               251.0|              641.0|                      0.0|
|         1051|ADULT INCONTINENCE|               0.75|               251.0|              983.0|                      1.0|
|         1066|ADULT INCONTINENCE|                0.0|               251.0|             1141.0|                      4.0|
|         1098|ADULT INCONTINENCE|                1.0|               251.0|              429.0|                      0.0|
|         1130|ADULT INC

### Creating training and test data

In [14]:
(training_norm,test_norm)=transformed_norm.randomSplit([0.8, 0.2])

### Creating ALS model and fitting data
- rating_label type must be numeric

### https://www.twblogs.net/a/5c2917d6bd9eee01606d2f58
### https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/563826/
### https://medium.com/@patelneha1495/recommendation-system-in-python-using-als-algorithm-and-apache-spark-27aca08eaab3

In [15]:
als=ALS(maxIter=5, # 算法迭代次數 maximum number of iterations
        regParam=0.09, #正則項權重
        rank=25, #模型中隱藏因子數目
        userCol="household_key_index", #column name for user ids. Ids must be (or can be coerced into) integers.
        itemCol="COMMODITY_DESC_index", #column name for item ids. Ids must be (or can be coerced into) integers.
        ratingCol="scaled_rating_label", #column name for ratings
        coldStartStrategy="drop", #drop any rows in the DataFrame of predictions that contain NaN values.
        # the model for new_user/new_prod,you can't predictions recommendation,
        # so you set drop you will drop the new_user/new_prod
        nonnegative=True) #商品推薦分數是否是非負的


model=als.fit(training_norm)

### Generate predictions and evaluate rmse

In [16]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="scaled_rating_label",predictionCol="prediction")

predictions=model.transform(test_norm)

rmse=evaluator.evaluate(predictions) # Evaluates the output with optional parameters.
print("RMSE="+str(rmse))

RMSE=0.21930986654355386


In [17]:
predictions.toPandas()

,household_key,COMMODITY_DESC,scaled_rating_label,COMMODITY_DESC_index,household_key_index,scaled_rating_label_index,prediction
0,108,PIES,1.00,148.0,496.0,0.0,0.772954
1,2297,PIES,1.00,148.0,2122.0,0.0,0.811888
2,1950,PIES,1.00,148.0,1721.0,0.0,0.794557
3,2453,PIES,0.50,148.0,85.0,3.0,0.743665
4,1845,PIES,0.25,148.0,65.0,2.0,0.746659
...,...,...,...,...,...,...,...
56819,2,SOAP - LIQUID & BAR,1.00,89.0,788.0,0.0,0.901642
56820,2198,SOAP - LIQUID & BAR,1.00,89.0,187.0,0.0,0.633591
56821,2475,SOAP - LIQUID & BAR,1.00,89.0,393.0,0.0,0.807157
56822,228,SOAP - LIQUID & BAR,1.00,89.0,954.0,0.0,0.845485


### Providing Recommendations

In [18]:
# Generate top 20 recommendations for each user

user_recs=model.recommendForAllUsers(20).show(10)

+-------------------+--------------------+
|household_key_index|     recommendations|
+-------------------+--------------------+
|               1580|[[3, 0.982944], [...|
|                471|[[3, 0.9273664], ...|
|               1591|[[3, 0.8198904], ...|
|               1342|[[3, 0.91405684],...|
|               2122|[[3, 0.99083936],...|
|               2142|[[3, 0.9785055], ...|
|                463|[[3, 0.9814789], ...|
|                833|[[3, 0.8232862], ...|
|               1645|[[3, 0.9160063], ...|
|                496|[[3, 0.94332737],...|
+-------------------+--------------------+
only showing top 10 rows



### Converting back to string form

In [19]:
import pandas as pd

# Generate top 10 recommendations for each user
recs=model.recommendForAllUsers(5).toPandas()


nrecs=recs.recommendations.apply(pd.Series) \
            .merge(recs, right_index = True, left_index = True) \
            .drop(["recommendations"], axis = 1) \
            .melt(id_vars = ['household_key_index'], value_name = "recommendation") \
            .drop("variable", axis = 1) \
            .dropna()
nrecs=nrecs.sort_values('household_key_index')

nrecs=pd.concat([nrecs['recommendation'].apply(pd.Series), nrecs['household_key_index']], axis = 1)
nrecs.columns = [
        
        'ProductID_index',
        'Rating',
        'UserID_index']

# raw data
md=transformed_norm.select(transformed_norm['household_key'],transformed_norm['household_key_index'],transformed_norm['COMMODITY_DESC'],transformed_norm['COMMODITY_DESC_index'])
md=md.toPandas()


dict1 =dict(zip(md['household_key_index'],md['household_key']))
dict2=dict(zip(md['COMMODITY_DESC_index'],md['COMMODITY_DESC']))


nrecs['household_key']=nrecs['UserID_index'].map(dict1)
nrecs['ProductID']=nrecs['ProductID_index'].map(dict2)
nrecs=nrecs.sort_values('household_key')
nrecs.reset_index(drop=True, inplace=True)


new=nrecs[['household_key','ProductID','Rating']]
#new['recommendations'] = list(zip(new.ProductID, new.Rating))
new['recommendations'] = list(new.ProductID)


res=new[['household_key','recommendations']]  
res_new=res['recommendations'].groupby([res.household_key]).apply(list).reset_index()
print(res_new)

     household_key                                    recommendations
0                1  [VEGETABLES - SHELF STABLE, SOUP, BAKED BREAD/...
1               10  [CHEESE, FLUID MILK PRODUCTS, SOUP, VEGETABLES...
2              100  [SOUP, VEGETABLES - SHELF STABLE, CHEESE, FLUI...
3             1000  [CHEESE, VEGETABLES - SHELF STABLE, BAKED BREA...
4             1001  [CHEESE, SOUP, BAKED BREAD/BUNS/ROLLS, VEGETAB...
...            ...                                                ...
2495           995  [BAKED BREAD/BUNS/ROLLS, FLUID MILK PRODUCTS, ...
2496           996  [FLUID MILK PRODUCTS, BAKED BREAD/BUNS/ROLLS, ...
2497           997  [CHEESE, SOUP, BAKED BREAD/BUNS/ROLLS, FLUID M...
2498           998  [FLUID MILK PRODUCTS, BAKED BREAD/BUNS/ROLLS, ...
2499           999  [SOUP, VEGETABLES - SHELF STABLE, CHEESE, FLUI...

[2500 rows x 2 columns]


/home/spark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
res_new.to_csv('./output_data/res_new_norm.csv',index=False)

In [26]:
def show_recommendation(household_key):
    if household_key not in res_new['household_key']:
        return('Customer not found.')
    else:
        recommendation_list = res_new[res_new['household_key'] == str(household_key)]['recommendations'].tolist()
        return [recommendation_list[0][i] for i in range(0,5)]
    

In [27]:
show_recommendation(1000)

['CHEESE',
 'VEGETABLES - SHELF STABLE',
 'BAKED BREAD/BUNS/ROLLS',
 'FLUID MILK PRODUCTS',
 'SOUP']

In [43]:
import pymysql
import csv
import sys

host = '3.113.29.214'  # '3.113.29.214'
user = 'eric'  # 'eric'
passwd = '123456'  # '123456'
port = 3306
conninfo = {'host' : host ,'port' : port,'user' : user , 'passwd' : passwd, 'db' : 'recommendation_system','charset' : 'utf8mb4'}


def add_csv(path):
    try:
        conn = pymysql.connect(**conninfo)
        cursor = conn.cursor()
        csv_data = csv.reader(open(path))
        cursor.execute("""DROP TABLE IF EXISTS als_1""")
        cursor.execute("""CREATE TABLE als_1 (household_key varchar(20), recommendations longtext)""")
        
        for row in csv_data:
            if csv_data.line_num == 1:
                continue
            cursor.execute('INSERT INTO als_1 (household_key, recommendations)'\
                           'VALUES("%s", "%s")',row)
        # close the connection to the database.
        conn.commit()
        print("Done")
    except:
        print('異常')
        print(sys.exc_info()[0])
        print(sys.exc_info()[1])
    finally:
        cursor.close()
        conn.close()
        print("db close")

In [44]:
add_csv('./output_data/res_new_norm.csv')

Done
db close


In [45]:
import pymysql
import csv
import sys

host = '3.113.29.214'  # '3.113.29.214'
user = 'eric'  # 'eric'
passwd = '123456'  # '123456'
port = 3306
conninfo = {'host' : host ,'port' : port,'user' : user , 'passwd' : passwd, 'db' : 'recommendation_system','charset' : 'utf8mb4'}


In [51]:
customer_id = 1000
try:
    conn = pymysql.connect(**conninfo)
    cursor = conn.cursor()
    try:
        recommendation = f"""select recommendations from als_2 where household_key = '{customer_id}'"""
        cursor.execute(recommendation)
        recommendation_item = cursor.fetchall()
        rec_list = []
        for i in recommendation_item[0][0].split(','):
            rec_list.append(i)
    except:
        print('error')
except:
    print('異常')
    print(sys.exc_info()[0])
    print(sys.exc_info()[1])
finally:
    cursor.close()
    conn.close()
    print("db close")


db close


In [52]:
item_list=rec_list

In [53]:
item_list

['CHEESE',
 ' VEGETABLES - SHELF STABLE',
 ' BAKED BREAD/BUNS/ROLLS',
 ' FLUID MILK PRODUCTS',
 ' SOUP']

In [55]:
rec_list = []
for i in item_list:
    rec_list.append(i)
try:
    conn = pymysql.connect(host='3.113.29.214', user='eric', password='123456', port=3306, \
                         database="store_db", charset='utf8mb4')
    cursor = conn.cursor()
    item_dict = {}
    for i in rec_list:
        i = i.strip()
        try:
            item = f"""select * from product where commodity = '{i}' ORDER BY RAND() LIMIT 1"""
            cursor.execute(item)
            recommendation_item = cursor.fetchall()
            item_dict[i] = recommendation_item
        except:
             item_dict[i] = 'no item record'
except:
    print('異常')
    print(sys.exc_info()[0])
    print(sys.exc_info()[1])
finally:
    cursor.close()
    conn.close()
    print("db close")

db close


In [56]:
item_dict

{'CHEESE': ((9420140,
   'Juvale Cheese Board Set - Charcuterie Board and Cheese Tools, Cheese and Meat Board, Includes 1 Bamboo Cutting Board and 4 Piece Knife Tools, 10.2" x 10.2" x 1.5"',
   20.99,
   'CHEESE',
   'https://m.media-amazon.com/images/I/918s5s6U3SL._AC_UL320_.jpg',
   None,
   184,
   4.6),),
 'VEGETABLES - SHELF STABLE': ((1005583,
   '72" Carson 5 Shelf Bookcase - Threshold&#153;',
   170.0,
   'VEGETABLES - SHELF STABLE',
   'https://target.scene7.com/is/image/Target/GUEST_3a9f6d41-d34e-42bf-8020-580e097985ce',
   'The Carson 5 Shelf Bookcase from Threshold is the perfect size and fit for any room in the home Made from durable wood this bookshelf features five spacious shelves for displaying your favorite books art and collectibles Clean modern lines make this piece easy to mix in with any style of decor whether in your bedroom living room or home office br br Depth at top of frame 13 6 br Overall width 30 6 br Overall height 72 br Overall depth at floor 12 2 br Dis